In [1]:
 import os
 arr = os.listdir("notes")
import glob

txtfiles = []
for file in glob.glob("./notes/*.md"):
    txtfiles.append(file)
    
arr = [md.replace(".md", "")for md in txtfiles]

In [2]:
arr = [md.replace("./notes/Q", "wd:Q")for md in arr]

In [10]:
readings = "{"
for i in arr:
    readings = readings + " " + i
readings = readings + " }"

print(readings)

{ wd:Q47579749 wd:Q24678240 wd:Q31194033 wd:Q29617753 wd:Q101226729 wd:Q90591926 wd:Q35830509 wd:Q28254676 wd:Q74657187 wd:Q42585641 wd:Q26707522 wd:Q56880438 wd:Q96817154 wd:Q52720499 wd:Q34169535 wd:Q2703462 wd:Q38798014 wd:Q37490532 wd:Q28974693 wd:Q57011346 wd:Q28181838 wd:Q30570244 wd:Q28292614 wd:Q33037209 wd:Q28200510 wd:Q29615252 wd:Q98665291 wd:Q98622035 wd:Q91997728 wd:Q34065481 wd:Q79763269 wd:Q38432885 wd:Q96032351 wd:Q101473819 wd:Q55483834 wd:Q21092566 wd:Q49483648 wd:Q78658697 wd:Q34878068 wd:Q86694255 wd:Q52374237 wd:Q42255083 wd:Q51427139 wd:Q59063943 wd:Q43299861 wd:Q36854753 wd:Q28250597 wd:Q40967362 wd:Q28185770 wd:Q100526347 wd:Q102205677 wd:Q93086975 wd:Q93086971 wd:Q21092584 wd:Q60137015 wd:Q48839876 wd:Q91130546 wd:Q90588777 wd:Q50735370 wd:Q47317210 wd:Q48094767 wd:Q59058260 wd:Q46412887 wd:Q33928865 wd:Q57001498 wd:Q99234140 }


In [18]:
query_1 = """

#defaultView:Table
SELECT
  (MIN(?dates) AS ?date)
  ?work ?workLabel
  (GROUP_CONCAT(DISTINCT ?type_label; separator=", ") AS ?type)
  (SAMPLE(?pages_) AS ?pages)
  ?venue ?venueLabel
  (GROUP_CONCAT(DISTINCT ?author_label; separator=", ") AS ?authors)
WHERE {
  VALUES ?work """ +  readings + """.
  ?work wdt:P50 ?author .
  OPTIONAL {
    ?author rdfs:label ?author_label_ . FILTER (LANG(?author_label_) = 'en')
  }
  BIND(COALESCE(?author_label_, SUBSTR(STR(?author), 32)) AS ?author_label)
  OPTIONAL { ?work wdt:P31 ?type_ . ?type_ rdfs:label ?type_label . FILTER (LANG(?type_label) = 'en') }
  ?work wdt:P577 ?datetimes .
  BIND(xsd:date(?datetimes) AS ?dates)
  OPTIONAL { ?work wdt:P1104 ?pages_ }
  OPTIONAL { ?work wdt:P1433 ?venue }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,no,ru,sv,zh". }  
}
GROUP BY ?work ?workLabel ?venue ?venueLabel
ORDER BY DESC(?date)  

"""

with open("sparql/authors.rq", "w+") as f:
  f.write(query_1)

In [20]:
query_2 = """

#defaultView:BubbleChart
SELECT ?score ?topic ?topicLabel
WITH {
  SELECT
    (SUM(?score_) AS ?score)
    ?topic
  WHERE {
        {
      SELECT (100 AS ?score_) ?topic WHERE {
        VALUES ?work """ +  readings + """.
         ?work  wdt:P921 ?topic . 
      }
    }
    UNION
    {
      SELECT (1 AS ?score_) ?topic WHERE {
        VALUES ?work """ +  readings + """.
        ?citing_work wdt:P2860 ?work .
        ?citing_work wdt:P921 ?topic . 
      }
    }
  }
  GROUP BY ?topic
} AS %results 
WHERE {
  INCLUDE %results
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,da,de,es,jp,no,ru,sv,zh". }
}
ORDER BY DESC(?score)
LIMIT 50

"""


with open("sparql/topic_bubble.rq", "w+") as f:
  f.write(query_2)

In [21]:
query_3 = """


#defaultView:Table
SELECT ?count ?theme ?themeLabel ?example_work ?example_workLabel
WITH {
  SELECT (COUNT(?work) AS ?count) ?theme (SAMPLE(?work) AS ?example_work)
  WHERE {
    VALUES ?work """ +  readings + """.
    ?work wdt:P921 ?theme .
  }
  GROUP BY ?theme
} AS %result
WHERE {
  INCLUDE %result
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh" . } 
}
ORDER BY DESC(?count) 

"""

with open("sparql/topic_table.rq", "w+") as f:
  f.write(query_3)

In [22]:
query_4 = """


# Venue statistics for a collection
SELECT
  ?count (SAMPLE(?short_name_) AS ?short_name)
  ?venue ?venueLabel
  ?topics ?topicsUrl
WITH {
  SELECT
    (COUNT(DISTINCT ?work) as ?count)
    ?venue
    (GROUP_CONCAT(DISTINCT ?topic_label; separator=", ") AS ?topics)
  WHERE {
    VALUES ?work """ +  readings + """.
    ?work wdt:P1433 ?venue .
    OPTIONAL {
      ?venue wdt:P921 ?topic .
      ?topic rdfs:label ?topic_label . FILTER(LANG(?topic_label) = 'en') }
  }
  GROUP BY ?venue
} AS %result
WHERE {
  INCLUDE %result
  OPTIONAL { ?venue wdt:P1813 ?short_name_ . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh". }  
} 
GROUP BY ?count ?venue ?venueLabel ?topics ?topicsUrl
ORDER BY DESC(?count)

"""

with open("sparql/venues.rq", "w+") as f:
  f.write(query_4)

In [24]:
query_5 = """


#defaultView:Map
SELECT ?organization ?organizationLabel ?geo ?count ?layer
WITH {
  SELECT DISTINCT ?organization ?geo (COUNT(DISTINCT ?work) AS ?count) WHERE {
    VALUES ?work """ +  readings + """.
        ?work wdt:P50 ?author .
    ?author ( wdt:P108 | wdt:P463 | wdt:P1416 ) / wdt:P361* ?organization . 
    ?organization wdt:P625 ?geo .
  }
  GROUP BY ?organization ?geo ?count
  ORDER BY DESC (?count)
  LIMIT 2000
} AS %organizations
WHERE {
  INCLUDE %organizations
  BIND(IF( (?count < 1), "No results", IF((?count < 2), "1 result", IF((?count < 5), "1 < results ≤ 10", IF((?count < 101), "10 < results ≤ 100", IF((?count < 1001), "100 < results ≤ 1000", IF((?count < 10001), "1000 < results ≤ 10000", "10000 or more results") ) ) ) )) AS ?layer )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }        
 }
ORDER BY DESC (?count)


"""


with open("sparql/author_map.rq", "w+") as f:
  f.write(query_5)

In [25]:
query_6 = """


#defaultView:Table
SELECT
  ?count
  ?citing_author ?citing_authorLabel

  # Either show the ORCID iD or construct part of a URL to search on the ORCID homepage
  (COALESCE(?orcid_, CONCAT("orcid-search/quick-search/?searchQuery=", ENCODE_FOR_URI(?citing_authorLabel))) AS ?orcid)
WITH {
  SELECT (COUNT(?citing_work) AS ?count) ?citing_author WHERE {
    VALUES ?work """ +  readings + """.
    ?citing_work wdt:P2860 ?work . 
    ?citing_work wdt:P50 ?citing_author .
  }
  GROUP BY ?citing_author 
  ORDER BY DESC(?count)
  LIMIT 500
} AS %counts
WITH {
  # An author might have multiple ORCID iDs
  SELECT
    ?count
    ?citing_author
    (SAMPLE(?orcids) AS ?orcid_)
  WHERE {
    INCLUDE %counts
    OPTIONAL { ?citing_author wdt:P496 ?orcids }
  }
  GROUP BY ?count ?citing_author
} AS %result
WHERE {
  INCLUDE %result
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh". }
} 
ORDER BY DESC(?count)


"""

with open("sparql/citing_authors.rq", "w+") as f:
  f.write(query_6)